## Basics of NumPy for scientific computing

### About

This is part of lecture notes of Math 104A *Introductory Numerical Analysis* course offered at the University of California Santa Barbara (Fall 2023). 
Author: Jea-Hyun Park

---
This work is licensed under [Creative Commons Attribution-ShareAlike 4.0 International](https://creativecommons.org/licenses/by-sa/4.0/)
Part of the content of this notebook is borrowed from the reference mentioned below. Thanks to all the authors sharing excellent knowledge.

### Reference

Main reference.

| Reference | Brief description |
|:---:|:---:|
| [NumPy quickstart](https://numpy.org/doc/stable/user/quickstart.html), </br>  [NumPy: the absolute basics for beginners](https://numpy.org/doc/stable/user/absolute_beginners.html#), </br> [NumPy fundamentals](https://numpy.org/doc/stable/user/basics.html) </br> (online document) | Tutorial offered by the official website of NumPy. |
|[Scientific Python Lectures](https://lectures.scientific-python.org/index.html) (online booklet) | Chapters 1.1-1.4 summarizes Python and NumPy. |

Some of the examples are inspired or borrowed from the following excellent video lecture. 

- [Introduction to Numerical Computing with NumPy | Alex Chabot-Leclerc](https://youtu.be/ZB7BZMhfPgk)

### Opening warning and advice

> ***Warning on libraries***
>
> - In this course, we will use **only** `numpy` (for computation) and `matplotlib` (for visualization). Unless there is a special need, sticking to these two does the best job for most scientific computing. 
> - In particular, **DO NOT USE** `sympy` (which is for symbolic mathematics) or `scipy` (higher-level scientific computing tool) in this course. 

Reasons

- Each library has their own philosophy and context that they target. Mixing them may well result in confusions if you don't have a complete knowledge. 
- Overly high-level tools do not suit the *analysis* aspect of 'numerical analysis'. You are expected to hand calculate simple mathematics. And **knowing what is really happening behind high-level tools is the goal** of this course!


> ***Advice***
>
> Do your best to use standard terminology. 

Everybody learns new things with impressions, images, and rough descriptions. But once you have grasped good enough ideas, use official names. It will pay off when you search further details and communicate with others about your issues or your ideas. There are so many similar-looking things that behave differently. For example, `list`, `tuple`, `dict`, and `numpy.ndarray` all contain many smaller things but work differently. If you ask for help with a bug saying 'I have this collection of numbers bla bla' or 'I created a vector bla bla', people may be able to understand the outline, but not what is the real issue with your code. Because there are many ways to contruct, say, a vector on computer. Instead, try to say 'I create a numpy array (i.e., `numpy.ndarray`) for a vector bla bla.' 

> ***Note***
>
> Use `numpy` array for scientific computing, NOT `list`. ([Learning objective]())


### Take-aways

After mastering this notebook, you will be able to
- use `numpy` array named `ndarray` for numerical tasks,
  - create and reshape arrays as you want,
  - clearly distinguish data types offered by NumPy,
- use `numpy` in an efficient way,
  - use `numpy` mathematical functions to manipulate `ndarray`s,
  - conduct element-wise operations of `ndarray`, maximizing **broadcasting**,
  - use NumPy's *fancy indexing*, *masking*, and `numpy.where` for delicate manipulation of arrays,
  - use NumPy's logical functions for fine control of computing,
- be aware possible issues,
  - be aware of auto-casting of data type,
  - be aware of change of dimensions when slicing or reshaping,
  - be aware that simple assignment does not copy the array and use `copy` method when necessary.


### Why `numpy.ndarray` and why not `list`?

| |`list`|`numpy.ndarray`|
|:--:|:--:|:--:|
| purpose | To contain general objects (`int`, `float`, `str`, `list` (nested), functions, etc.) | To compute collections of numbers (`int`, `float`, `complex`, `bool`) |
| speed | slow | fast | 

See the following for speed.

In [1]:
"""Experiment: speed of vector additions with `numpy.ndarray` and `list`
"""
import numpy as np
from time import time

#=== parameters
N = 10**3 # size of vector
T = 10**3 # number of additions carried out
a_lst = [i for i in range(N)] 
a = np.arange(N)
b = 1

#=== `numpy` addition [0, 1, 2, ..., N-1] + [1, 1, 1, ..., 1] 
start = time()
for _ in range(T):
    a + b
end = time()
t = end - start
print(f"NumPy: {T} additions takes {t} (sec)")

#=== `list` addition [0, 1, 2, ..., N-1] + [1, 1, 1, ..., 1]
start = time()
for _ in range(T):
    for i in range(N):
        a_lst[i] + b
end = time()
t = end - start
print(f"List : {T} additions takes {t} (sec)")

NumPy: 1000 additions takes 0.001856088638305664 (sec)
List : 1000 additions takes 0.08910012245178223 (sec)


### Creation

- `numpy.zeros(shape: tuple)`
- `numpy.ones(shape: tuple)`
- `numpy.arange`: start is inclusive and stop is **exclusive**.
  - `numpy.arange(n)`: ndarray of [0, 1, 2, ..., n - 1] (if `n` is integer).
  - `numpy.arange(m, n)`: ndarray of [m, m+1, m+2, ..., n - 1] (if `m` and `n` are integer).
  - `numpy.arange(m, n, k)`: ndarray of [m, m+k, m+2k, ..., n - 1] (if `m`, `n`, and `k` are integer).
- `numpy.linspace(start, end, n)`: ndarray of equally spaced `n` real numbers between `start` and `end` **inclusively**. (`n` must an integer.)

In [2]:
import numpy as np

# zeros
a = np.zeros((3,4))
print(a)


[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]


In [3]:

# ones
b = np.ones((4,2))
print(b)


[[1. 1.]
 [1. 1.]
 [1. 1.]
 [1. 1.]]


In [4]:

# arange(start, end); 'start' inclusive, 'end' EXCLUSIVE
c = np.arange(5) # eqv to: np.arange(0,5)
d = np.arange(4, 9) 
print(c)
print(d)


[0 1 2 3 4]
[4 5 6 7 8]


In [5]:

# linspace(start, end, #of pts); 'start' and 'end' BOTH inclusive
e = np.linspace(0, 1, 9)
print(e)


[0.    0.125 0.25  0.375 0.5   0.625 0.75  0.875 1.   ]


### size, shape, reshape, ndim

Given an ndarray `arr`,
- `arr.size`, `arr.shape`, and `arr.ndim` are the most important *information* (or *properties/attributes*) of `arr`.

|`arr.size`| `arr.shape`| `arr.ndim`|
|:-:|:-:|:-:|
| total number of entries (integer) | rectangular shape of an array (tuple) | dimension of the array (integer) |

- `arr.reshape(new_shape: tuple)` is a very useful *feature* (or *method*). 
  - One component of the shape can omitted. NumPy's convention is to put `-1` to have NumPy decide what's omitted.

In [6]:
# size vs shape
A = np.arange(12)
print("A: ", A)
print("A (ndim):", A.ndim)
print("A (size): ", A.size)
print("A (shape): ", A.shape)


A:  [ 0  1  2  3  4  5  6  7  8  9 10 11]
A (ndim): 1
A (size):  12
A (shape):  (12,)


In [7]:

# reshape (not in-place)
print(A.reshape(3, 4))
print("A (size after reshape): ", A.size)
print("A (shape after reshape): ", A.shape)
print("A (after reshape): ", A)


[[ 0  1  2  3]
 [ 4  5  6  7]
 [ 8  9 10 11]]
A (size after reshape):  12
A (shape after reshape):  (12,)
A (after reshape):  [ 0  1  2  3  4  5  6  7  8  9 10 11]


In [8]:

# reshape and copy
B = A.reshape((3,4))
print("B: ", B)
print("B (size): ", B.size)
print("B (shape): ", B.shape)


B:  [[ 0  1  2  3]
 [ 4  5  6  7]
 [ 8  9 10 11]]
B (size):  12
B (shape):  (3, 4)


In [10]:

# transpose (not in-place)
C = B.T
print(f"C looks like: \n{C}")
print(C.shape)
print(B)


C looks like: 
[[ 0  4  8]
 [ 1  5  9]
 [ 2  6 10]
 [ 3  7 11]]
(4, 3)
[[ 0  1  2  3]
 [ 4  5  6  7]
 [ 8  9 10 11]]


In [11]:

# 'lazy' reshape: one slot of shape = '-1' --> auto-computed
D = C.reshape((1,-1)) 
print(f"C looks like: \n{C}")
print(f"D looks like: \n{D}")
print("D (shape): ", D.shape)


C looks like: 
[[ 0  4  8]
 [ 1  5  9]
 [ 2  6 10]
 [ 3  7 11]]
D looks like: 
[[ 0  4  8  1  5  9  2  6 10  3  7 11]]
D (shape):  (1, 12)


In [12]:

# ndim (number of dimensions)
print("A:", A)
print("D:", D)
print("dimension of A:", A.ndim)
print("dimension of D:", D.ndim)


A: [ 0  1  2  3  4  5  6  7  8  9 10 11]
D: [[ 0  4  8  1  5  9  2  6 10  3  7 11]]
dimension of A: 1
dimension of D: 2


**Question**

How would you change the terminology in the summary above?

- *information*
- *feature*

Type two words in order separated by a comma. And give a reason for that.

This is **about atmosphere**, not getting it right.
1. Think for a short time.
2. Share your guess with your pair.
3. Type your answer in clicker.
4. Feel free to say out loud.


### Operations and functions

> ***Note***
>
> Every operation and function works component-wisely.

> ***Note***
>
> Use `numpy` math functions, ***NOT*** `math` module. [Learning objective]()

`numpy` functions are faster and better compatible with `numpy.ndarray`.

In [ ]:
import numpy as np

# Every operation and function works component-wisely
a = np.linspace(1, 12, 12)
b = np.arange(-6, 6)
print("a = ", a)
print("b = ", b)
print("a + b = ", a + b)
print("a - b = ", a - b)
print("a * b = ", a * b)
print("a ^ b = ", a ** b)
print("sin(b) = ", np.sin(b)) 
# Do not use math.sin(b)

### Broadcasting

![broadcasting1](https://numpy.org/doc/stable/_images/broadcasting_1.png)

![broadcasting2](https://numpy.org/doc/stable/_images/broadcasting_4.png)

Figure source: NumPy documentation (https://numpy.org/doc/stable/user/basics.broadcasting.html)

- The above two are the most frequent use cases. If you want to use more sophisticated applications, see the documentation https://numpy.org/doc/stable/user/basics.broadcasting.html.
- Broadcasting is not only convenient, but it is also fast. 

> **Broadcasting rules**
> 
> The length of each dimension of two arrays must either **match or one of them is 1**. NumPy then "stretches" the block of length 1 to match the size, and carry out the operations.

> ***Note***
>
> Broadcasting is an important skill for *vectorized* programming. This is one of the good programming practices (Learning objectives), especially in scientific computing. It may take some getting-used-to, but never tried to avoid it. 

In [ ]:
import numpy as np

# broadcating a constant
a = np.arange(4)
print("matrix (or a vector) + constant")
x = a + 3
print(a, " + ", 3, "=", x)


In [ ]:

# broadcasting between a row and a column
b = np.arange(4).reshape((-1,1))
c = np.arange(3)
print("\ncolumn + row")
print(b, "+", c, "=")
# print(c)
print(b+c)


In [ ]:

# broadcasting between a matrix and a column
b = np.arange(12).reshape((-1,3))
c = np.arange(4).reshape((4,-1)) * 100 
print("\nmatrix + column")
print(f"{b}\n   +   \n{c}    \n   =   ")
print(b+c)

##### Speed test: broadcasting vs full matrix

In [ ]:
from time import time

T = 10**3
N = 10**3
L = 100
xx = np.linspace(0, L, N)
yy = np.linspace(0, L, N).reshape((-1,1))
XX, YY = np.meshgrid(xx, yy)

start = time()
for _ in range(N):
    zz = np.sin(xx) + 2*yy
end = time()
print("time taken by broadcasting: ", end - start, "(sec)")

start = time()
for _ in range(N):
    ZZ = np.sin(XX) + 2*YY
end = time()
print("time taken by full matrix:  ", end - start, "(sec)")


### Slicing

#### Summary

- We can access and modify part of an array using slicing. 
- The syntax is the same as slicing of `list`: `start:end:jump`.
    * `start` is inclusive; default = 0 (the very first).
    * `end` is exclusive; default = length of the array (all the way to the end).
    * `jump` default = 1.
    * Negative integers can be used to index from backwards, including `jump`.

#### Slicing 1D arrays

In [ ]:
import numpy as np

a = np.arange(6)
print(a[:])
print(a[1:-1])
print(a[::2])
print(a[1::2])
print(a[::-1])


In [ ]:

# multiply even-indexed entried by a specific array
a[1::2] = a[1::2] * [3, 5, 7]
print(a)


In [ ]:

# replace odd-indexed entries with -1
a[::2] = -1 # broadcasting in effect
print(a)

#### Slicing 2D arrays

In [ ]:
import numpy as np

a = np.arange(25).reshape((5,5))
print(a)
print(a[1::2, ::2])


In [ ]:

# replace entries using slicing
a[2] = 0
print(a)

a[2, :] = a[-1, :]
print(a)

### Fancy indexing, masking, `where`, and `piecewise`

#### Fancy indexing

This is best described by examples. See below.

In [ ]:
import numpy as np

# extract entries using collection of indices
a = np.arange(18).reshape((3,6))
ind_ori = ([0, 1, 1, 2, 2], # collection of row index
       [1, 2, -1, 4, 5]) # collection of col index
b = a[ind_ori] # pick only (row, col)-entries for each pair 
print(a)
print(b)


In [ ]:

# We can modify entries using collection of indices
a[ind_ori] = -99
print(a)


#### Masking: Indexing by Booleans

Masking literally *masks* an array so that the operation applies only to a certain entries. This is done by passing boolean arrays as index. See the example below.

In [ ]:
import numpy as np

# Using masking, we can modify only the entries that satisfy a particular condition
a = np.arange(18).reshape((3,6))
print(a)
# create a mask
mask = (a > 5) & (a < 10) # for Python, use `and`
print(mask)
a[mask] = -1
print(a)


In [ ]:

# Once you become comfortable, use more compact version
b = np.arange(18).reshape((3,6))
b[(b>5) & (b<10)] = -1
print(b)

##### Misc masking

In [ ]:
import numpy as np

a = np.arange(25).reshape((5, 5))
print(a)
print(a % 4)
print(a[a % 4 == 0])

#### `where`

In a nutshell, `numpy.where` function does similar jobs to masking.

- We can use `where` to find indices of certain entries.
- `where` can be used to apply an operation only to certain entries. 
  - `where(condition array, array for true, array for false)` (see example below)
  - `array for true` and `array for false` must be broadcastable to the condition array: `where` picks values from them.

In [ ]:
import numpy as np

a = np.array([[1, 2, 5, 2, 5], 
              [3, 5, 2, 1, 3]])

# this gives a mask where max occurs
mask = (a == a.max()) 
print(a)
print(mask)
print(a[mask])


In [ ]:

# `where` gives a tuple of index arrays
ind_ori = np.where(a == a.max())
print(ind_ori)
print(a[ind_ori]) # this is basically fancy indexing


##### Implementation of piecewise functions

###### `numpy.piecewise` function

- Syntax: `numpy.piecewise(x, list_of_masks, list_of_fns)`
- `list_of_masks` and `list_of_fns` must have the same length, or `list_of_fns` has one more element. In this case, the last one applies for all other cases that are not specified by the masks.
- `list_of_masks` must have the same size of `x`.

Using `numpy.piecewise`

$$ y = \begin{cases}
        x & (0 \le x \le 5)
        \\
        -1 & (5 < x < 10)
        \\
        5x & (\text{otherwise})
        \end{cases}
$$

In [ ]:
import numpy as np

xx = np.arange(18).reshape((3,6))

masks = [(xx >= 0) & (xx <= 5), (xx < 10) &  (xx > 5)] #, (xx >= 10) & (xx <= 17)]
fns = [lambda x: x, lambda x: -1, lambda x: 5*x]

yy = np.piecewise(xx, masks, fns)

print("xx\n", xx)
print("\nyy\n", yy)

Implementation of using `numpy.where`
$$ y = \begin{cases}
        -1 & (5 < x < 10)
        \\
        5x & (\text{otherwise})
        \end{cases}
$$

In [ ]:
# piece-wise operation using `where`
x = np.arange(18).reshape((3,-1))
y = np.where((x > 5) & (x < 10), -1, 5 * x) 
print(x)
print(-1*np.ones((3,6)))
print(5*x)
print(y)

### Mathematical functions

- `max`, `min`
- `argmin`, `argmax`: returns the **linear index** where the first max or min occurs. 
  - In case of multiple occurrences of the minimum values, the indices corresponding to the first occurrence are returned.
- `unravel_index`: takes linear index and shape and returns shape index.
    - It is useful to obtain multi-dimensional index of `argmin` and `argmax`
    - `ind = np.unravel_index(a.argmin(), a.shape)`, then `a[ind] == a.min()`.
- `ptp`: finds (max - min). (ptp stands for 'peak to peak')
- If you need random real numbers, use `np.random.rand` function, which returns uniform random numbers between 0 and 1 in the shape passed in. 

In [ ]:
import numpy as np
a = np.random.randint(1, 20, (3,4))
# lin_ind = a.argmin()
ind_ori = np.unravel_index(a.argmin(), a.shape)
print(a)
print(a.argmin())
print(ind_ori)
print(a[ind_ori] == a.min())

print(a.max(), a.min(), np.ptp(a))


More math functions

- `numpy.sin`, `numpy.cos`, `numpy.exp`, `numpy.log`, `numpy.sqrt`, `numpy.power` = `**`, `numpy.abs`, `numpy.round`, `numpy.floor`, `numpy.ceil`, etc. 
- `numpy.diff` are useful for different quotients or numerical differentiation and integration. 
- For a complete list, see [documentation](https://numpy.org/doc/stable/reference/routines.math.html)
- Constants: `numpy.pi`, `numpy.e`, `numpy.nan` (standing for 'not a number'; usually resulting from dividing by 0), `numpy.inf`

In [ ]:
import numpy as np

x = np.arange(5)
sqrt_x = np.sqrt(x)
print(x)
print(sqrt_x)

In [ ]:

print(np.log(x))
print(0/x)


In [ ]:

print(np.round(sqrt_x, 2))
print(np.floor(sqrt_x))
print(np.pi)

### Logical functions

- `numpy.allclose`: very useful to check if computations are correct (see below).
- `numpy.all` (big 'and'), `numpy.any` (big 'or')
- `numpy.isfinite`, `numpy.isnan`, `numpy.isinf`: useful to check any pathological results.
- `numpy.logical_and` = `&`, `numpy.logical_or` = `|`, `numpy.logical_not`, `numpy.logical_xor` (only one of two is true)

In [ ]:
import numpy as np

x = np.arange(5)
sqrt_x = np.sqrt(x)

print(x ** (1/2))
print(sqrt_x)
print(np.allclose(x ** (1/2), sqrt_x))
print(x ** (1/2) == sqrt_x)

In [ ]:

print(np.log(x))
print(np.isfinite(np.log(x)))


In [ ]:

bigger_than_2 = x > 2
less_than_4 = x < 4
print(x)


In [ ]:

print(bigger_than_2)
print(np.all(bigger_than_2))


In [ ]:

print(less_than_4) 
print(np.any(less_than_4))
print(bigger_than_2 & less_than_4)

### Care needed

#### Auto-casting of data type

In [ ]:
import numpy as np

a = np.arange(5)
print(a)
print(a.dtype)

# even if float is feeded, a truncate decimals
a[0] = 1.2222
print(a, "(data type):", a.dtype) 

#### Dimension changes when slicing

When slicing, dimensions may change.

- The first two actions below lower the dimension of a 2D array to 1D array.
  * `a[1, :]` is more explicit that `a` is a at least two dimensional array.
  * Hence, it will raise an error if `a` is changed to a 1D array. This may be a good thing because it raises an error early (easier to debug).
  * `a[1]` works even if `a` is changed to a 1D array.
  * But, its dimension is not clear from this expression.
- There are a number of alternatives that preserve the shape. See the examples below. 
  - But their behavior may be different than what you expect. So, the key is to be **aware of this kind of behavoirs**.

In [ ]:
import numpy as np
import sys
# include a path to import personaltools.py module
sys.path.append('../') 
from personaltools.numpyrelatedtools import check_np_dim

a = np.arange(8).reshape((2,4))
print(a)


In [ ]:

# The following two actions lower the 2D array to 1D array
b = a[1] # second row of a
check_np_dim(b, 'b')

c = a[1, :] # second row of a
check_np_dim(c, 'c')


In [ ]:

# possible alternative to preserve the shape
# 1. reshape
d = a[1, :].reshape((1,-1))
check_np_dim(d, 'd')


In [ ]:

# 2. manually increase the dimension 
e = a[1, None]
check_np_dim(e, 'e')

f = a[1, np.newaxis]
check_np_dim(f, 'f')


In [ ]:

# 3. use list index or array index
g = a[[1], :]
check_np_dim(g, 'g')

h = a[np.array([1]), :]
check_np_dim(h, 'h')

# But be careful. The following looks similar, but gives different result.
i = a[[1], [2, 3]]
check_np_dim(i, 'i')


#### Copy arrays

- When slicing an array, a new array that points to the same memory address is created. Therefore, any modification affects the original array. 
- This is to save computation for allocate memory and copy contents.
- Solution to this is `copy()` method.
- The same applies to `reshape`.

> ***Note***
>
> Slicing works with reference (address of memory).


In [ ]:
import numpy as np

a = np.arange(6)
b = a[::2]
print('a = ', a)
print('b = ', b)


In [ ]:

# modify b
b[0] = -99.
print('a = ', a)
print('b = ', b)


In [ ]:

# copy and modify
a = np.arange(6)
c = a[::2].copy()
c[0] = -99
print('a = ', a)
print('c = ', c)


In [ ]:

# copy and modify with `reshape` method
A = np.arange(6)
B = A.reshape((3,2))
print('A = ', A)
print('B = ', B)


In [ ]:

# modify b
B[-1,-1] = -99.
print('A = ', A)
print('B = ', B)


In [ ]:
# copy and modify
A = np.arange(6)
C = A.reshape((3,2)).copy()
C[-1,-1] = -99
print('A = ', A)
print('C = ', C)


##### Slicing of list and array

- `list` make a copy when sliced while
- `ndarray` does not make a copy when sliced
  - (advanced) `ndarray` only changes *stride* when sliced for efficiency.

In [ ]:
import numpy as np

lst = [i for i in range(8)]
arr = np.arange(8)
print("lst: ", lst)
print("arr: ", arr)


In [ ]:

lst2 = lst[::2]
arr2 = arr[::2].copy()

lst2[0] = -99
arr2[0] = -99


In [ ]:

print("lst2 (sliced and modifed): ", lst2)
print("arr2 (sliced and modified): ", arr2)

print("lst (after 'copy' is modified): ", lst)
print("arr (after 'copy' is modified): ", arr)

### Advanced

#### Row-major in high-dimensional arrays

- As the dimension of an array increases, the new dimension prepends, called **row-major**. [Video](https://youtu.be/ZB7BZMhfPgk?t=4734)
- If `dim = (2, 3, 4, 5)` is the shape of an array, then `dim(-1)` is always the column vector dimension. (Again, column vector is displayed as a row.) This is what is used in `axis` argument. (See [Array computation rules](#array-computation-rules))
- C is also row-major.
- Matlab and Fortran is column major, where the new dimensions append.

In [ ]:
import numpy as np

n1, n2, n3, n4 = 2, 3, 4, 5
a = np.arange(n1)
print(a)

a = np.arange(n1*n2).reshape((n1,n2))
print(a)
print(a[0, :])
print(a[:, 1])

# 3D array is two 3X4 matrices
# in Matlab, this is four 2X3 matrices
a = np.arange(n1*n2*n3).reshape((n1, n2, n3))
print(a) 
# first 3X4 matrix
print(a[0, :, :]) 
# 2X4 matrix: all entires with 2nd slot index 1
# dimension decreases
print(a[:, 1, :]) 
# 2X3 matrix: all entires with 3nd slot index 2
# dimension decreases
print(a[:, :, 2]) 

#### Array computation rules

1. Operations between multiple array objects are first checked for proper shape match (*broadcasting*).
1. All operations and functions are element-wise.
2. Reduction operations (`numpy.mean`, `numpy.std`, `numpy.sum`, `numpy.prod`, `numpy.max`, `numpy.min`, etc) apply to the whole array, unless an `axis` is specified.
    - For example, `np.sum(a, axis = 0)` means 'sum entries of array `a` **along** axis 0 (the outer most index according to *row-major* rule)'. Therefore, the axis feeded will go away.
    - `axis = -1` always sum along the inner most index, i.e. it aggregates each column vectors. (See [Row-major high-dimensional arrays](#row-major-high-dimensional-arrays))
3. Missing values (`nan`, stands for 'not a number') propagate unless explicitely ignored (`numpy.nanmean`, `numpy.nansum`, etc).

In [ ]:
import numpy as np

# rule 4: nan propagate unless using 
a = np.array([np.nan, 1, 3, 5])
print(a)
print(np.mean(a)) # take average including nan's
print(np.nanmean(a)) # take average excluding nan's

#### Reshape and transpose


##### Reshape and transpose behave differently

In [4]:
import numpy as np

# reshape vs transpose
a_orig = np.arange(5*3, dtype=np.float64).reshape((5,3))

a_reshaped = a_orig.reshape(3,5)
a_transposed = a_orig.T

print("a original (5,3)\n", a_orig)
print("a reshaped to (3,5)\n", a_reshaped)
print("a transposed to (3,5)\n", a_transposed)

a original (5,3)
 [[ 0.  1.  2.]
 [ 3.  4.  5.]
 [ 6.  7.  8.]
 [ 9. 10. 11.]
 [12. 13. 14.]]
a reshaped to (3,5)
 [[ 0.  1.  2.  3.  4.]
 [ 5.  6.  7.  8.  9.]
 [10. 11. 12. 13. 14.]]
a transposed to (3,5)
 [[ 0.  3.  6.  9. 12.]
 [ 1.  4.  7. 10. 13.]
 [ 2.  5.  8. 11. 14.]]


##### Reshaped data access: linear index

> **Summary**
>
> `arr_ori[ind1]==arr_reshaped[ind2]` iff linear index of `ind1` = `ind2`


- The original array and the new array has the same data iff their linear indices are the same.
- How to get multidimensional index from a linear index: `multi_dim_ind = np.unravel_index(lin_index, shape)`.
- How to get linear index from a multidimensional index: `lin_ind = np.ravel_multi_index(multi_int, shape)`.
  - These can be used to conduct mutiple such tasks by passing `dim=` parameter. (See the documentation for detail.)
  - These can do the *column-major* version by passing `order=` parameter. (See the documentation for detail.)

In [12]:
import numpy as np

# create an array with random entries, and its reshape
arr_ori = np.random.rand(2, 3, 5, 7)
arr_reshaped = arr_ori.reshape((3, 5, 2, 7))

# Two indices for each array: These are chosen to be linear index (or flattened index) = 122 (less than 2*3*5*7 = 210)
ind_ori = (1,0,2,3)
ind_reshaped = (1,3,1,3)

# print the values of the two indices
print("original:", ind_ori, arr_ori[ind_ori])
print("reshaped:", ind_reshaped, arr_reshaped[ind_reshaped])

# print the linear index of the two indices
print("linear index of original array:", np.ravel_multi_index(ind_ori, (2,3,5,7)))
print("linear index of reshaped array:", np.ravel_multi_index(ind_reshaped, (3,5,2,7)))


original: (1, 0, 2, 3) 0.17226201704659516
reshaped: (1, 3, 1, 3) 0.17226201704659516
linear index of original array: 122
linear index of reshaped array: 122


##### Transpose data access: permutation

- 1D array: `transpose` does NOTHING.
- 2D array: `transpose` behaves the same way as matrix transpose.
- High-diminsional array: Use permutation (usually a `tuple` of indices) to specify transpositions.
  - Supposing we have named `perm` our permutation as in the example below, `perm` represents the *resulting* dimensions in terms of the origianl dimensions: `perm[i]`-th dimension of the new array = i-th dimension of the original array. (See the summary below.)

> **Summary**
> 
> Given `new = old.transpose(perm)`, we have
> 
> - `new.shape[i] == old.shape[perm[i]] for i in range(old.ndim)` 
> - `old[ind_old] == new[ind_new]` iff `np.all([ind_new[i]==ind_old[perm[i]] for i in range(old.ndim)])` 

In [34]:
import numpy as np

# create an array with random entries, and its reshape
arr_ori = np.random.rand(2, 3, 5, 7)

# permutation of dimensions for transpose
# new.shape[i] = old.shape[perm[i]] for i = 0, 1, 2, 3
perm = (2, 0, 3, 1)
arr_transposed = arr_ori.transpose(perm)

# check the shape of the two arrays
print([arr_transposed.shape[i] == arr_ori.shape[perm[i]] for i in range(arr_ori.ndim)])

# Two indices using a permutation of dimensions
ind_ori = (1,0,-1,5)
ind_transposed = (-1,1,5,0)

# print the values of the two indices
print("original:", ind_ori, arr_ori[ind_ori])
print("reshaped:", ind_transposed, arr_transposed[ind_transposed])
print("Entry equality condition:", np.all([ind_transposed[i]==ind_ori[perm[i]] for i in range(arr_ori.ndim)]))

[True, True, True, True]
[True, True, True, True]
original: (1, 0, -1, 5) 0.02619612663819626
reshaped: (-1, 1, 5, 0) 0.02619612663819626
Entry equality condition: True


We can make the index conversion as a function to reuse conveniently.

In [36]:
def permute_index(ind_ori, perm):
    """Permute the index according to the permutation of dimensions

    INPUTS:
    -------
    ind_ori: tuple
        original index
    perm: tuple
        permutation of dimensions
    OUTPUTS:
    --------
    ind_permuted: tuple
        permuted index
    """
    ind_permuted = tuple([ind_ori[perm[i]] for i in range(len(ind_ori))])
    return ind_permuted

ind_transposed = permute_index(ind_ori, perm) # using a function

# check the index matching of the two arrays
print([ind_transposed[i] == ind_ori[perm[i]] for i in range(arr_ori.ndim)])

# print the values of the two indices
print("original:", ind_ori, arr_ori[ind_ori])
print("reshaped:", ind_transposed, arr_transposed[ind_transposed])
print("Entry equality condition:", np.all([ind_transposed[i]==ind_ori[perm[i]] for i in range(arr_ori.ndim)]))

[True, True, True, True]
original: (1, 0, -1, 5) 0.02619612663819626
reshaped: (-1, 1, 5, 0) 0.02619612663819626
Entry equality condition: True


**Inversion of transpose**

- Apply the inverse permutation.
- NumPy offers `argsort` for this purpose (see the example below).
- More explicitly, the inverse permutation is obtained by creating an array with its `perm[i]`-th entry being `i` back as its original had: `inv_perm[perm[i]]=i` for `i range in len(perm)`. A vectorized version reads: `inv_perm = np.zeros(len(perm), dtype=np.int64); inv_perm[perm] = arange(len(perm));`.
- (Wrong way) Applying the same permutation twice results in something different since a permutation squared is not the identity permutation except for some special cases (with multipication being interpreted as compositions).

> **Summary** (inversion of transposition)
> 
> Given `perm` is used for transposition,
>
> - `arr_ori == arr_transposed.transpose(inv_perm)` iff `inv_perm == np.argsort(perm)`

In [49]:
# inverting transposition using `argsort`, manual, vectorized manual
inv_perm = np.argsort(perm)
inv_perm_manual = (1,3,0,2)
inv_perm_manual2 = np.zeros(len(perm), dtype=np.int64)
inv_perm_manual2[np.array(perm)] = np.arange(len(perm))

print("original shape:", arr_ori.shape)
print("transposed shape:", arr_transposed.shape)
print("(WRONG inversion) transposing twice using the same permutation:", arr_transposed.transpose(perm).shape)
print("back to origianl (argsort):", arr_transposed.transpose(np.argsort(perm)).shape)
print("back to origianl (manual):", arr_transposed.transpose(inv_perm_manual).shape)

print("Inverse permutation condition check:", np.all(inv_perm_manual == np.argsort(perm)))
print("Inverse permutation condition check:", np.all(inv_perm_manual == inv_perm_manual2))

original shape: (2, 3, 5, 7)
transposed shape: (5, 2, 7, 3)
(WRONG inversion) transposing twice using the same permutation: (7, 5, 3, 2)
back to origianl (argsort): (2, 3, 5, 7)
back to origianl (manual): (2, 3, 5, 7)
Inverse permutation condition check: True
Inverse permutation condition check: True


### Appendix

#### Test: `where` vs mask for piece-wise functions

In [ ]:
import numpy as np
from time import time

T = 10**3
N = 10**3
L = 100
xx = np.linspace(0, L, N)
yy = np.linspace(0, L, N)
xx, yy = np.meshgrid(xx, yy, indexing='ij')

fns = [[lambda x,y: x+y, lambda x,y: x**2 + yy**2],
        [lambda x,y: x-y, lambda x,y: x**2 - yy**2]]
masks = [[(xx > 0.5) * (yy > 0.5), (xx > 0.5) * (yy <= 0.5)],
         [(xx <= 0.5) * (yy > 0.5), (xx <= 0.5) * (yy <= 0.5)]]


start = time()
for _ in range(N):
    zz = np.where(masks[0][0], fns[0][0](xx, yy), fns[0][1](xx, yy))
end = time()
print("time taken by where:                     ", end - start, "(sec)")

start = time()
for _ in range(N):
    zz = masks[0][0]*fns[0][0](xx, yy) + (~ masks[0][0]) * fns[0][1](xx, yy)
end = time()
print("time taken by multiplication by bools:   ", end - start, "(sec)")


#### Test: `piecewise` vs mask for piece-wise functions

In [ ]:
import numpy as np
from time import time

T = 10**4
N = 10**4
L = 100.0
xx = np.linspace(0, L, N)

fns = [lambda x: 2*x, lambda x: x**2, lambda x: x**3, lambda x: np.sin(x)]
masks = [(xx < L/4) &  (xx >= 0), (xx < L/2) &  (xx >= L/4), (xx < 3*L/2) &  (xx >= L/2), (xx >= 3*L/4)]


start = time()
for _ in range(T):
    zz = np.piecewise(xx, masks, fns)
end = time()
print("time taken by piecewise:                 ", end - start, "(sec)")

start = time()
for _ in range(T):
    zz = masks[0]*fns[0](xx) + masks[1]*fns[1](xx) + masks[2]*fns[2](xx) + masks[3]*fns[3](xx)
end = time()
print("time taken by multiplication by bools:   ", end - start, "(sec)")


#### Test: Vectorized vs `for`-loop

Task: Blurring an image



In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from time import time
from personaltools.numpyrelatedtools import check_np_dim

img = plt.imread('images/citystreet.jpg')

# vectorized blurring
start = time()

# top = img[:-2, 1:-1, :]
# bottom = img[2:, 1:-1, :]
# left = img[1:-1, :-2, :]
# right = img[1:-1, 2:, :]
# center = img[1:-1, 1:-1, :]

# apply average many times
blurred_img1 = img[:, :]

for i in range(1):
    top = blurred_img1[:-2, 1:-1]
    bottom = blurred_img1[2:, 1:-1]
    left = blurred_img1[1:-1, :-2]
    right = blurred_img1[1:-1, 2:]
    center = blurred_img1[1:-1, 1:-1]

    blurred_img1 = (top + bottom + left + right + center)/5

end = time()

fig, ax = plt.subplots(1,3, figsize=(15, 30))
ax[0].imshow(img)
ax[1].imshow(blurred_img1)

print("time taken by vectorized code:   ", end - start, "(sec)")
check_np_dim(img, 'img', suppress_arr=True)

if True:
    start = time()
    Nrow, Ncol = img.shape
    blurred_img2 = np.zeros((Nrow - 2, Ncol - 2))
    for i in range(Nrow - 2):
        for j in range(Ncol - 2):
            # blurred = (center + top + bottom + left + right)/5
            blurred_img2[i, j] = (img[i+1, j+1] + img[i, j+1] + img[i+2, j+1] + img[i+1, j] + img[i+1, j+2])/5
    end = time()
    print("time taken by for-loop:          ", end - start, "(sec)")
    ax[2].imshow(blurred_img2)

#### For Matlab users

If you are proficient in Matlab and want to leverage that skills for NumPy, the following will help.

- [NumPy for MATLAB users (from NumPy documentation)](https://numpy.org/doc/stable/user/numpy-for-matlab-users.html)

- [Thesaurus of Mathematical Languages,
or MATLAB synonymous commands in Python/NumPy](https://mathesaurus.sourceforge.net/)

#### Not recommended but just in case

##### Implementation of piece-wise function of three cases using `numpy.where`

I recommend using masking for the following implementation. But anyway, it is possible.

$$ y = \begin{cases}
        100 & (0 \le x \le 5)
        \\
        -1 & (x < 5 < 10)
        \\
        5x & (\text{otherwise})
        \end{cases}
$$

In [ ]:

# piece-wise operation using `where`
x = np.arange(18).reshape((3,-1))
y = np.where((x > 5) & (x < 10), -1, 
        np.where((x >= 0) & (x <=5), -100, 5 * x))
print(x)
print(y)

In [ ]:
# Why is the previous code true?
print("     Case: false")
print(np.where((x >= 0) & (x <=5), -100, 5 * x))
print("     Case: true")
print(np.where((x > 5) & (x < 10), -1,5 * x))
print("     Combined")
print(np.where((x > 5) & (x < 10), -1,
        np.where((x >= 0) & (x <=5), -100, 5 * x)))


#### PyTorch and NumPy: Differences

PyTorch, which is a popular tool for implementing neural networks, is also used as a scientific computing. The developers of PyTorch have emulated NumPy. But there are some different conventions for the same or similar functionalities. The primary reason for the change of names appears to make things more suggestive of what they really give.

Setting: `arr` is a numpy array and `tsr` is a torch Tensor.

| | NumPy | PyTorch |
|:-:|:-:|:-:|
| main object | `array` (`numpy.array`) | `tensor` (`torch.tensor`) |
| total dimensionality | `arr.ndim` | `arr.ndim` |
| organization of array | `arr.shape` | `tsr.size()`|
| total number of entries | `arr.size` | `tsr.numel()`|

In [8]:
import torch as tc
import numpy as np

arr = np.arange(5)
tsr = tc.from_numpy(arr)

print("numpy arr: ", arr)
print("  - type: ", type(arr))
print("  - ndim: ", arr.ndim)
print("  - shape: ", arr.shape)
print("  - size: ", arr.size)

print("torch tsr: ", tsr)
print("  - type: ", type(tsr))
print("  - ndim: ", tsr.ndim)
print("  - shape: ", tsr.shape)
print("  - size(): ", tsr.size())
print("  - size: ", tsr.size)
print("  - nelm: ", tsr.numel())


numpy arr:  [0 1 2 3 4]
  - type:  <class 'numpy.ndarray'>
  - shape:  (5,)
  - size:  5
  - ndim:  1
torch tsr:  tensor([0, 1, 2, 3, 4])
  - type:  <class 'torch.Tensor'>
  - shape:  torch.Size([5])
  - size():  torch.Size([5])
  - size:  <built-in method size of Tensor object at 0x7fc2213a5950>
  -nelm:  5
  - ndim:  1
